In [1]:
import ujson
import pandas as pd
import json
try:
  import jsonlines
except ImportError:
  !pip install jsonlines
  import jsonlines
  
try:
    from tqdm import tqdm
except ImportError:
    !pip install tqdm
    import tqdm

try:
  import adjustText
except ModuleNotFoundError:
  !pip install adjustText

import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict
from torch.nn import RNNCell
from torch.nn import RNNBase, RNN
from torch.nn import Embedding
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import adjustText
! pip install umap
import umap
import matplotlib.pyplot as plt
from adjustText import adjust_text
import torch.nn.functional as F
import random


with open('/Users/user/Downloads/complaints.json', 'r') as f:
    dat = ujson.load(f)

  Using cached umap-0.1.1-py3-none-any.whl


In [2]:
zip_codes_demographic_info = pd.read_csv('../data/Demographic_Statistics_By_Zip_Code.csv')
zip_codes_demographic_info.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   JURISDICTION NAME                    236 non-null    int64  
 1   COUNT PARTICIPANTS                   236 non-null    int64  
 2   COUNT FEMALE                         236 non-null    int64  
 3   PERCENT FEMALE                       236 non-null    float64
 4   COUNT MALE                           236 non-null    int64  
 5   PERCENT MALE                         236 non-null    float64
 6   COUNT GENDER UNKNOWN                 236 non-null    int64  
 7   PERCENT GENDER UNKNOWN               236 non-null    int64  
 8   COUNT GENDER TOTAL                   236 non-null    int64  
 9   PERCENT GENDER TOTAL                 236 non-null    int64  
 10  COUNT PACIFIC ISLANDER               236 non-null    int64  
 11  PERCENT PACIFIC ISLANDER        

In [3]:
print(len(dat)) # 1906322 complaints

1906322


In [5]:
# covid date
nyc_zipcodes = zip_codes_demographic_info['JURISDICTION NAME'].map(str).dropna().unique().tolist()
print(len(nyc_zipcodes))

236


In [6]:
complaints_raw_df =pd.DataFrame(dat)
print(complaints_raw_df.columns)
print(complaints_raw_df.shape)
complaints_raw_df.head()

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'complaint_what_happened', 'company_public_response', 'company',
       'state', 'zip_code', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response', 'timely',
       'consumer_disputed', 'complaint_id'],
      dtype='object')
(1906322, 18)


,date_received,product,sub_product,issue,sub_issue,complaint_what_happened,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response,timely,consumer_disputed,complaint_id
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,,TRANSWORLD SYSTEMS INC,FL,335XX,,Consent provided,Web,2019-09-24,Closed with explanation,Yes,N/A,3384392
1,2020-12-27,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,AL,35401,,Consent not provided,Web,2020-12-27,Closed with explanation,Yes,N/A,4032788
2,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,N/A,3379500
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",,"Diversified Consultants, Inc.",NC,275XX,,Consent provided,Web,2019-11-08,Closed with explanation,Yes,N/A,3433198
4,2020-12-27,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MS,39110,,Consent not provided,Web,2020-12-27,Closed with explanation,Yes,N/A,4032769


In [7]:
complaints_raw_df['product'].dropna().unique()

array(['Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Checking or savings account', 'Credit card or prepaid card',
       'Money transfer, virtual currency, or money service',
       'Vehicle loan or lease', 'Mortgage', 'Student loan',
       'Payday loan, title loan, or personal loan', 'Credit card',
       'Consumer Loan', 'Payday loan', 'Bank account or service',
       'Credit reporting', 'Other financial service', 'Money transfers',
       'Prepaid card', 'Virtual currency'], dtype=object)

In [7]:
# statistical tests 

In [8]:
date_cols = ['date_received', 'date_sent_to_company', ]
id_cols = ['complaint_id']
_cols = ['state', 'zip_code']


In [22]:
# complaints = complaints_raw_df[(complaints_raw_df['state']=='NY') & (complaints_raw_df['complaint_what_happened']!='')]
# len(complaints)
nyc_zipcodes

['10001',
 '10002',
 '10003',
 '10004',
 '10005',
 '10006',
 '10007',
 '10009',
 '10010',
 '10011',
 '10012',
 '10013',
 '10014',
 '10016',
 '10017',
 '10018',
 '10019',
 '10020',
 '10021',
 '10022',
 '10023',
 '10024',
 '10025',
 '10026',
 '10027',
 '10028',
 '10029',
 '10030',
 '10031',
 '10032',
 '10033',
 '10034',
 '10035',
 '10036',
 '10037',
 '10038',
 '10039',
 '10040',
 '10044',
 '10045',
 '10065',
 '10115',
 '10119',
 '10128',
 '10154',
 '10278',
 '10280',
 '10301',
 '10302',
 '10303',
 '10304',
 '10305',
 '10306',
 '10307',
 '10309',
 '10310',
 '10312',
 '10314',
 '10451',
 '10452',
 '10453',
 '10454',
 '10455',
 '10456',
 '10457',
 '10458',
 '10459',
 '10460',
 '10461',
 '10462',
 '10463',
 '10464',
 '10465',
 '10466',
 '10467',
 '10468',
 '10469',
 '10471',
 '10472',
 '10473',
 '10474',
 '10475',
 '10514',
 '10543',
 '10553',
 '10573',
 '10701',
 '10705',
 '10911',
 '10965',
 '10977',
 '11001',
 '11021',
 '11050',
 '11101',
 '11102',
 '11103',
 '11104',
 '11105',
 '11106',


In [9]:

complaints_raw_df['OlderAmerican'] = complaints_raw_df['tags'].map(lambda x: True if 'Older American' in x else False)
complaints_raw_df['ServiceMember'] = complaints_raw_df['tags'].map(lambda x: True if 'Service' in x else False)
complaints_raw_df['NoSpecialTagPerson'] = complaints_raw_df['tags'].map(lambda x:True if x == '' else False)
complaints_raw_df[['OlderAmerican', 'ServiceMember', 'NoSpecialTagPerson']].describe()

,OlderAmerican,ServiceMember,NoSpecialTagPerson
count,1906322,1906322,1906322
unique,2,2,2
top,False,False,True
freq,1789474,1756511,1660119


In [10]:
complaints_raw_df['company_public_response'].unique()

array(['',
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes the complaint is the result of a misunderstanding',
       'Company believes complaint is the result of an isolated error',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company disputes the facts presented in the complaint',
       "Company can't verify or dispute the facts in the complaint",
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company chooses not to provide a public response',
       'Company believes complaint relates to a discontinued policy or procedure',
       "Company believes the complaint provided an opportunity to answer consumer's questions"],
      dtype=object)

In [11]:
complaints_raw_df['date'] = pd.to_datetime(complaints_raw_df['date_received'])

In [12]:
from datetime import datetime
complaints_raw_df['YEAR'] = complaints_raw_df['date_received'].map(lambda x: x.split('-')[0])
complaints_raw_df['YEAR'].value_counts()

2020    432579
2019    277373
2018    257302
2017    242959
2016    191466
2015    168474
2014    153043
2013    108217
2012     72373
2011      2536
Name: YEAR, dtype: int64

In [34]:
complaints_raw_df_filtered = complaints_raw_df[(complaints_raw_df['state']=='NY')&(complaints_raw_df['YEAR'].isin(['2019', '2021'])) & (complaints_raw_df['complaint_what_happened'] != '')&(complaints_raw_df['zip_code']!='')]
print(complaints_raw_df_filtered.shape)


(5724, 23)


In [35]:
nyc_zipcodes_firsthree = [a[:3] for a in nyc_zipcodes]

In [39]:
nyc_complaints_df = complaints_raw_df_filtered[complaints_raw_df_filtered['zip_code'].map(lambda x: True if x[:3] in nyc_zipcodes_firsthree else False)]

In [40]:
nyc_complaints_df.shape

(5013, 23)

In [42]:
nyc_complaints_df

,date_received,product,sub_product,issue,sub_issue,complaint_what_happened,company_public_response,company,state,zip_code,...,date_sent_to_company,company_response,timely,consumer_disputed,complaint_id,OlderAmerican,ServiceMember,NoSpecialTagPerson,date,YEAR
10,2019-07-08,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,Hello This complaint is against the three cred...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,109XX,...,2019-07-08,Closed with explanation,Yes,N/A,3299394,False,False,True,2019-07-08,2019
18,2019-10-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,i am a victim of identity theft as previously ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,10035,...,2019-10-24,Closed with explanation,Yes,N/A,3415907,False,False,True,2019-10-24,2019
87,2019-11-18,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,The following accounts do NOT BELONG TO ME! Th...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,115XX,...,2019-11-18,Closed with explanation,Yes,N/A,3442257,False,False,True,2019-11-18,2019
407,2019-02-21,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,I have sent several certified letters to Profe...,,Professional Bureau of Collections of Maryland...,NY,104XX,...,2019-02-21,Closed with explanation,Yes,N/A,3159318,False,False,True,2019-02-21,2019
419,2019-02-20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,XX/XX/2018 Transunion received from me a certi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,104XX,...,2019-02-20,Closed with explanation,Yes,N/A,3157997,False,False,True,2019-02-20,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905410,2019-02-18,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I recently reviewed my credit report and found...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,112XX,...,2019-02-18,Closed with explanation,Yes,N/A,3155362,False,False,True,2019-02-18,2019
1905525,2019-02-20,Credit card or prepaid card,Store credit card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,In XX/XX/XXXX I was contacted by a con artist ...,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,NY,100XX,...,2019-02-21,Closed with explanation,Yes,N/A,3158145,True,False,False,2019-02-20,2019
1905547,2019-02-15,Credit card or prepaid card,Store credit card,Problem when making payments,Problem during payment process,XXXX XXXX XXXX RE : Regarding a Reported Delin...,,Alliance Data Card Services,NY,104XX,...,2019-02-15,Closed with explanation,Yes,N/A,3153617,False,False,True,2019-02-15,2019
1905655,2019-02-12,Checking or savings account,Checking account,Closing an account,Funds not received from closed account,On XX/XX/XXXX I have noticed all my funds has ...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",NY,100XX,...,2019-02-12,Closed with explanation,Yes,N/A,3149079,False,False,True,2019-02-12,2019


In [ ]:
{'': '__label__noresponse',
    'Company has responded to the consumer and the CFPB and chooses not to provide a public response': '__label__',
       'Company believes complaint represents an opportunity for improvement to better serve consumers': '__label__',
       'Company believes the complaint is the result of a misunderstanding': '__label__',
       'Company believes complaint is the result of an isolated error': '__label__',
       'Company believes it acted appropriately as authorized by contract or law': '__label__',
       'Company disputes the facts presented in the complaint': '__label__',
       "Company can't verify or dispute the facts in the complaint": '__label__',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company chooses not to provide a public response': '__label__',
       'Company believes complaint relates to a discontinued policy or procedure': '__label__',
       "Company believes the complaint provided an opportunity to answer consumer's questions": '__label__'
}

In [20]:
complaints_raw_df['company_response'].unique()

array(['Closed with explanation', 'Closed with non-monetary relief',
       'In progress', 'Untimely response', 'Closed with monetary relief',
       'Closed', 'Closed with relief', 'Closed without relief', ''],
      dtype=object)

In [43]:
pd.crosstab(complaints_raw_df['OlderAmerican'], complaints_raw_df['ServiceMember'])

ServiceMember,False,True
OlderAmerican,,
False,1660119,129355
True,96392,20456


In [ ]:
def load_wikitext(filename='wikitext2-sentencized.json'):
    if not os.path.exists(filename):
        !wget "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json" -O $filename
    
    datasets = json.load(open(filename, 'r'))
    for name in datasets:
        datasets[name] = [x.split() for x in datasets[name]]
    vocab = list(set([t for ts in datasets['train'] for t in ts]))      
    print("Vocab size: %d" % (len(vocab)))
    return datasets, vocab

def perplexity(model, sequences):
    n_total = 0
    logp_total = 0
    for sequence in sequences:
        logp_total += model.sequence_logp(sequence)
        n_total += len(sequence) + 1  
    ppl = 2 ** (- (1.0 / n_total) * logp_total)
    return ppl
  
def perplexity_printcounter(model, sequences):
    n_total = 0
    logp_total = 0
    counter = 0
    for sequence in sequences:
        counter += 1
        if counter%500 == 0:
          print(counter)
        logp_total += model.sequence_logp(sequence)
        n_total += len(sequence) + 1  
    ppl = 2 ** (- (1.0 / n_total) * logp_total)
    return ppl

In [23]:
complaints2 = complaints_raw_df
NY = complaints2[(complaints2['state']=='NY') ].sort_values('date_received', ascending=False)
NY['date_received'] = pd.to_datetime(NY['date_received'])
subset_2019_20 = NY[NY['YEAR'].isin(['2019', '2020'])]#shape # 65,555
subset_complaints = subset_2019_20[subset_2019_20['complaint_what_happened']!=''].drop_duplicates('complaint_what_happened')

print(subset_complaints.shape) # 23130
subset_complaints # 

(13491, 22)


,date_received,product,sub_product,issue,sub_issue,complaint_what_happened,company_public_response,company,state,zip_code,...,submitted_via,date_sent_to_company,company_response,timely,consumer_disputed,complaint_id,OlderAmerican,ServiceMember,date,YEAR
588320,2020-12-23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I am writing to delete the following informati...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,117XX,...,Web,2020-12-23,Closed with explanation,Yes,N/A,4027599,False,False,2020-12-23,2020
543272,2020-12-23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I'VE HAD A CHANCE TO LOOK OVER MY CREDIT REPOR...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,112XX,...,Web,2020-12-23,Closed with explanation,Yes,N/A,4027396,False,False,2020-12-23,2020
602094,2020-12-23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,AFTER RECEIVING A CURRENT COPY OF MY CREDIT RE...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,11428,...,Web,2020-12-23,Closed with explanation,Yes,N/A,4027567,False,False,2020-12-23,2020
604201,2020-12-23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,The new credit card I received and have been u...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,146XX,...,Web,2020-12-23,Closed with explanation,Yes,N/A,4027619,False,False,2020-12-23,2020
602104,2020-12-23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I contacted you before about these accounts on...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NY,114XX,...,Web,2020-12-23,Closed with explanation,Yes,N/A,4027669,False,False,2020-12-23,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18187,2019-01-01,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,Previous complaint sent regarding issues with ...,,"EQUIFAX, INC.",NY,114XX,...,Web,2019-01-01,Closed with explanation,Yes,N/A,3114016,False,False,2019-01-01,2019
904038,2019-01-01,Debt collection,Credit card debt,False statements or representation,Attempted to collect wrong amount,Capital one bank kept increased my credit card...,,CAPITAL ONE FINANCIAL CORPORATION,NY,112XX,...,Web,2019-01-01,Closed with explanation,Yes,N/A,3113872,False,False,2019-01-01,2019
1060564,2019-01-01,Checking or savings account,Checking account,Managing an account,Problem accessing account,Ally bank. I opened a account and deposited a ...,Company has responded to the consumer and the ...,ALLY FINANCIAL INC.,NY,11213,...,Web,2019-01-01,Closed with non-monetary relief,Yes,N/A,3114006,False,False,2019-01-01,2019
871419,2019-01-01,Mortgage,Conventional home mortgage,Closing on a mortgage,,I applied in XX/XX/XXXX with Key Bank. XX/XX/X...,,KEYCORP,NY,12050,...,Web,2019-01-03,Closed with explanation,Yes,N/A,3113835,True,False,2019-01-01,2019


In [26]:
new_df = subset_complaints.reset_index()

In [28]:
# Import Counter
# artcile


article = new_df['complaint_what_happened'][0]
from nltk import word_tokenize
from collections import Counter

# Tokenize the article: tokens
tokens = word_tokenize(article)

# Convert the tokens into lowercase: lower_tokens
lower_tokens = [t.lower() for t in tokens]
print(lower_tokens)
# Create a Counter with the lowercase tokens: bow_simple
bow_simple = Counter(lower_tokens)

# Print the 10 most common tokens
print(bow_simple.most_common(10))

['i', 'am', 'writing', 'to', 'delete', 'the', 'following', 'information', 'in', 'my', 'file', '.', 'the', 'items', 'i', 'need', 'deleted', 'are', 'listed', 'in', 'the', 'report', '.', 'i', 'am', 'a', 'victim', 'of', 'identity', 'theft', 'and', 'did', 'not', 'make', 'the', 'charge', '.', 'i', 'ask', 'that', 'the', 'items', 'be', 'deleted', 'to', 'correct', 'my', 'credit', 'report', '.', 'i', 'reported', 'the', 'theft', 'of', 'my', 'identity', 'to', 'the', 'federal', 'trade', 'commission', 'and', 'i', 'also', 'have', 'enclosed', 'copies', 'of', 'the', 'federal', 'trade', 'commissions', 'identity', 'theft', 'affidavit', '.', 'please', 'delete', 'the', 'items', 'as', 'soon', 'as', 'possible', '.']
[('the', 9), ('i', 6), ('.', 6), ('to', 3), ('my', 3), ('items', 3), ('of', 3), ('identity', 3), ('theft', 3), ('am', 2)]


In [29]:

def get_zip_codes(val, zip_codes=['100', '101','102']):
    for zc in zip_codes:
        if val.startswith(zc):
            return True
        
subset_complaints['nyc'] = subset_complaints['zip_code'].map(lambda x: True if len(get_match(x, ['100', '101', '102'])) > 0 else False)

In [1]:
subset_complaints

NameError: name 'subset_complaints' is not defined

In [38]:


subset_complaints['len'] = subset_complaints['complaint_what_happened'].map(lambda x: len(x))
subset_complaints[(subset_complaints['nyc']==True)].to_excel('nyc.xlsx') #.sort_values(['len', 'date_received'], ascending=[False, False])['complaint_what_happened'].tolist()


In [28]:
import pandas as pd
#Data Analyses Libraries
import pandas as pd                
import numpy as np    
from urllib.request import urlopen
import json
import glob
import os

#Importing Data plotting libraries
import matplotlib.pyplot as plt     
import plotly.express as px       
import plotly.offline as py       
import seaborn as sns             
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import matplotlib.ticker as ticker
import matplotlib.animation as animation

#Other Miscallaneous Libraries
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML
import matplotlib.colors as mc
import colorsys
from random import randint
# Import libraries
import os, sys, gc, re
import time
import datetime
import jsonlines
import numpy as np
import pandas as pd
import xmltodict as xtd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
from xml.etree import ElementTree as etree
import io
import zipfile
from collections import OrderedDict
import string
from IPython.display import display, HTML
from io import StringIO
from bs4 import BeautifulSoup
import ujson
import glob
import grequests
import jsonlines
import gzip
import requests

import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from collections import defaultdict 
from pathlib import Path
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")
pd.options.plotting.backend = "plotly"



def NullUnique(df):
    dic = defaultdict(list)
    for col in df.columns:
        dic['Feature'].append(col)
        dic['NumUnique'].append(len(df[col].unique()))
        dic['NumNull'].append(df[col].isnull().sum())
        dic['%Null'].append(round(df[col].isnull().sum()/df.shape[0] * 100,2))
    return pd.DataFrame(dict(dic)).sort_values(['%Null'],ascending=False).style.background_gradient()


def get_age(age_str):
    if 'month' in age_str.lower():
        return np.float(returnNonList(get_match(age_str, ['\d+'])))/12
    
    elif 'year' in age_str.lower():
        return np.float(returnNonList(get_match(age_str, ['\d+'])))
    
    else:
        return returnNonList(get_match(age_str, ['\d+']))

                             
def read_jsonl(file_path):
    """Read a .jsonl file and yield its contents line by line.
    file_path (unicode / Path): The file path.
    YIELDS: The loaded JSON contents of each line.
    """
   # cnt = 0
    with Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            # cnt += 1
            try:  # hack to handle broken jsonl
                yield ujson.loads(line.strip())
            except ValueError:
                continue

# adding simple warning filter
warnings.simplefilter('ignore')

from tqdm.notebook import tqdm_notebook


dev_rex_list = ['NCT[ ]?\d{8}']

def get_match(text, rex):
    if isinstance(rex, (list, tuple, set)):
        rex = '(' + '|'.join(rex) + ')'
    result = re.findall(rex, text)
    return result


def unorder_dict(x):
    if isinstance(x, OrderedDict):
        return {k: unorder_dict(v) for k,v in x.items()}
    elif isinstance(x, list):
        return [unorder_dict(v) for v in x]
    else:
        return x

    

def isinList(val, list_):
    if val in list_:
        return 1
    else:
        return 0

def create_count_bucket(val):
    """creates bucket based on count"""
    if val > 0:
        return 1
    else:
        return 0

    
def read_delim(map_file, sep='\t'):
    """fxn to read in delim files"""
    return pd.read_csv(map_file, sep=sep)




def flatten_json(y):
    """
    this flattens_json obj
    adds begining key to beginning
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out


def cleanAge(age):
    if len(re.findall(r'\(.*\)',age)):
        return re.findall(r'\(.*\)',age)[0]
    return '('+age+')'



def returnNonList(val):
    if not isinstance(val, list):
        return val
    if isinstance(val, list):
        if len(val) == 1:
            return val[0]
        else:
            return '|'.join(val)
        

transtable = str.maketrans(dict.fromkeys(string.punctuation))

def to_string(s):
    """
    takes input s and returns a string type and encoding if necessary
    """
    try:
        return str(s)
    except:
        # Change the encoding type if needed
        return s.encode('utf-8')


def unicodeToAscii(s):
    """
    input: string
    output: string ascii
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        )


def strip_punctuation(input_string):
    """
    cleans string by stripping punctuation
    """

    return input_string.translate(transtable)



def create_df_from_table(table):
    """takes a bs4 soup obj and returns a list of dicts"""
    l = {}
    rows = iter(table)
    for row in rows:
        values = [col.text for col in row]

        l[values[0]] = values[1]
    return l


def replace_none(val, returnval = '--'):
    if val == None:
        return returnval
    
    else:
        return val
    
    
def read_json(fin):
    with open(fin, 'r') as f:
        dat = ujson.load(f)
    return dat


def return_soup_from_html(path_counts):
    """returns beautfiul soup obj """
    with open(path_counts, 'r') as fin:
        soup = BeautifulSoup(fin, 'html.parser')
    return soup

today = datetime.datetime.today().strftime('%Y-%m-%d')
print(f"{today}")

def chunks(l, n):
    """Yield n number of sequential chunks from l."""
    d, r = divmod(len(l), n)
    for i in range(n):
        si = (d + 1) * (i if i < r else r) + d * (0 if i < r else i - r)
        yield l[si:si + (d + 1 if i < r else d)]


flatten = lambda l: [item for sublist in l for item in sublist]

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)


2021-03-07
Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'complaint_what_happened', 'company_public_response', 'company',
       'state', 'zip_code', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response', 'timely',
       'consumer_disputed', 'complaint_id'],
      dtype='object')
(128850, 18)


In [9]:
complaints2['text'] = complaints2['issue'] + '.   ' + complaints2['complaint_what_happened'] + '.  ' + complaints2['company_public_response']

complaints2['text'].map(lambda x: x.replace(''))

0          Attempts to collect debt not owed   transworld...
1          Incorrect information on your report     Compa...
2          Incorrect information on your report     Compa...
3          Communication tactics   Over the past 2 weeks,...
4          Incorrect information on your report     Compa...
                                 ...                        
1906317    Loan servicing, payments, escrow account   On ...
1906318    Loan servicing, payments, escrow account   I c...
1906319    Transaction issue   I made a payment to CITI X...
1906320    Credit reporting company's investigation   cfb...
1906321    Incorrect information on credit report   My hu...
Name: text, Length: 1906322, dtype: object

In [10]:
complaints.company_public_response.value_counts()

                                                                                                                           1112140
Company has responded to the consumer and the CFPB and chooses not to provide a public response                             627353
Company believes it acted appropriately as authorized by contract or law                                                     83669
Company chooses not to provide a public response                                                                             52473
Company believes the complaint is the result of a misunderstanding                                                            7658
Company disputes the facts presented in the complaint                                                                         6741
Company believes complaint caused principally by actions of third party outside the control or direction of the company       4769
Company believes complaint is the result of an isolated error                      

In [35]:
filtered_complaints = complaints[(complaints['company_public_response']!='')]
filtered_complaints = filtered_complaints[(filtered_complaints['complaint_what_happened']!='')]
filtered_complaints.shape

(322764, 18)

In [36]:
filtered_complaints['company_public_response'].value_counts()

Company has responded to the consumer and the CFPB and chooses not to provide a public response                            246728
Company believes it acted appropriately as authorized by contract or law                                                    40136
Company chooses not to provide a public response                                                                            19818
Company believes the complaint is the result of a misunderstanding                                                           3861
Company disputes the facts presented in the complaint                                                                        3649
Company believes complaint is the result of an isolated error                                                                2522
Company believes complaint caused principally by actions of third party outside the control or direction of the company      2502
Company believes complaint represents an opportunity for improvement to better serve consu

In [37]:
filtered_complaints['product'].value_counts(sort=True)

Credit reporting, credit repair services, or other personal consumer reports    137889
Debt collection                                                                  53829
Mortgage                                                                         37510
Credit card or prepaid card                                                      23009
Credit reporting                                                                 18985
Checking or savings account                                                      14969
Credit card                                                                       9361
Bank account or service                                                           8627
Vehicle loan or lease                                                             4005
Student loan                                                                      3985
Consumer Loan                                                                     3631
Payday loan, title loan, or personal loan  

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def return_polarity(sentence):
    if isinstance(sentence, str):
        vs = analyzer.polarity_scores(sentence)





0          Attempts to collect debt not owed   transworld...
1          Incorrect information on your report     Compa...
2          Incorrect information on your report     Compa...
3          Communication tactics   Over the past 2 weeks,...
4          Incorrect information on your report     Compa...
                                 ...                        
1906317    Loan servicing, payments, escrow account   On ...
1906318    Loan servicing, payments, escrow account   I c...
1906319    Transaction issue   I made a payment to CITI X...
1906320    Credit reporting company's investigation   cfb...
1906321    Incorrect information on credit report   My hu...
Name: text, Length: 1906322, dtype: object

In [39]:
filtered_complaints['tags'].value_counts()

                                 267850
Servicemember                     34201
Older American                    16378
Older American, Servicemember      4335
Name: tags, dtype: int64

In [40]:
filtered_complaints['consumer_consent_provided'].value_counts()

Consent provided    322764
Name: consumer_consent_provided, dtype: int64

In [46]:
filtered_complaints['sub_issue'].value_counts()

                                                               69101
Information belongs to someone else                            42003
Their investigation did not fix an error on your report        25593
Account status incorrect                                       13093
Account information incorrect                                  12662
                                                               ...  
Problem with a check written from your prepaid card account        5
Trouble using the card to send money to another person             4
Threatened to turn you in to immigration or deport you             2
Was signed up for overdraft on card, but don't want to be          2
Qualified for a better loan than the one offered                   1
Name: sub_issue, Length: 217, dtype: int64

In [54]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    #note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
    #from vaderSentiment import SentimentIntensityAnalyzer

# --- examples -------
fcomplaints = set()
for fc in filtered_complaints['sub_issue'].dropna().unique():
    if fc == '':
        continue
    elif fc.startswith(' '):
        continue
    else:
        fcomplaints.add(fc)
print(len(fcomplaints))

analyzer = SentimentIntensityAnalyzer()
for sentence in fcomplaints:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs['neg'])))


216
Non-sufficient funds and associated fees------------------------- 0.0
Problem with rewards from credit card---------------------------- 0.237
Attempted to/Collected exempt funds------------------------------ 0.0
Used obscene/profane/abusive language---------------------------- 0.0
Charged for a purchase or transfer you did not make with the card 0.13
Application denied----------------------------------------------- 0.744
Keep getting calls about your loan------------------------------- 0.0
Credit denial---------------------------------------------------- 0.0
Their investigation did not fix an error on your report---------- 0.0
Sued you in a state where you do not live or did not sign for the debt 0.135
Problem using a debit or ATM card-------------------------------- 0.31
Problem getting your free annual credit report------------------- 0.214
Not given enough info to verify debt----------------------------- 0.294
Denied request to lower payments--------------------------------- 0.6

In [57]:
filtered_complaints['complaint_what_happened']

9          Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...
10         Hello This complaint is against the three cred...
14         Today XX/XX/XXXX went online to dispute the in...
17         Please reverse the late payments reported on t...
18         i am a victim of identity theft as previously ...
                                 ...                        
1906312    I just logged on to a credit site to join and ...
1906313    I had called Citibank on XXXX/XXXX/XXXX around...
1906319    I made a payment to CITI XXXX Credit Card on X...
1906320    cfbp i would Like to file a complaint on Exper...
1906321    My husband and I are in the middle of an FHA S...
Name: complaint_what_happened, Length: 322764, dtype: object

In [6]:
import en_core_web_lg
nlp = en_core_web_lg.load()



In [7]:
sents = filtered_complaints['complaint_what_happened'].dropna().tolist()


NameError: name 'filtered_complaints' is not defined

In [70]:
sents = [sent.replace('\n', ' ') for sent in sents]

In [72]:
docs = nlp.pipe(sents[:100])
ldocs = list(docs)

In [73]:
entsdf = pd.DataFrame([doc.to_json() for doc in ldocs])
entsdf

,text,ents,sents,tokens
0,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...","[{'start': 15, 'end': 25, 'label': 'ORG'}, {'s...","[{'start': 0, 'end': 46}, {'start': 46, 'end':...","[{'id': 0, 'start': 0, 'end': 10, 'pos': 'ADV'..."
1,Hello This complaint is against the three cred...,"[{'start': 36, 'end': 41, 'label': 'CARDINAL'}...","[{'start': 0, 'end': 69}, {'start': 70, 'end':...","[{'id': 0, 'start': 0, 'end': 5, 'pos': 'INTJ'..."
2,Today XX/XX/XXXX went online to dispute the in...,[],"[{'start': 0, 'end': 127}]","[{'id': 0, 'start': 0, 'end': 5, 'pos': 'NOUN'..."
3,Please reverse the late payments reported on t...,"[{'start': 70, 'end': 79, 'label': 'PERSON'}]","[{'start': 0, 'end': 79}, {'start': 80, 'end':...","[{'id': 0, 'start': 0, 'end': 6, 'pos': 'INTJ'..."
4,i am a victim of identity theft as previously ...,[],"[{'start': 0, 'end': 52}]","[{'id': 0, 'start': 0, 'end': 1, 'pos': 'PRON'..."
...,...,...,...,...
95,XXXX XXXX -- -- I am not aware of this debt I ...,"[{'start': 0, 'end': 9, 'label': 'PERSON'}]","[{'start': 0, 'end': 15}, {'start': 16, 'end':...","[{'id': 0, 'start': 0, 'end': 4, 'pos': 'PROPN..."
96,I am writing this complaint due to the lack of...,"[{'start': 86, 'end': 90, 'label': 'ORG'}, {'s...","[{'start': 0, 'end': 122}, {'start': 123, 'end...","[{'id': 0, 'start': 0, 'end': 1, 'pos': 'PRON'..."
97,There's a collection showing on my credit in t...,"[{'start': 61, 'end': 68, 'label': 'MONEY'}, {...","[{'start': 0, 'end': 106}, {'start': 107, 'end...","[{'id': 0, 'start': 0, 'end': 5, 'pos': 'PRON'..."
98,"To Whom It May Concern, The reason for filing ...","[{'start': 131, 'end': 160, 'label': 'ORG'}, {...","[{'start': 0, 'end': 246}, {'start': 247, 'end...","[{'id': 0, 'start': 0, 'end': 2, 'pos': 'PART'..."


In [77]:
import cov_bsv

nlp = cov_bsv.load()
doc = nlp('Pt tested for COVID-19. His wife was recently diagnosed with novel coronavirus. SARS-COV-2: Detected')

print(doc.ents)
print(doc._.cov_classification)
cov_bsv.visualize_doc(doc)

ModuleNotFoundError: No module named 'cov_bsv'

In [82]:
from spacy import displacy
from IPython.core.display import display, HTML

html = displacy.render(ldocs[0], style="ent")
display(HTML(html))


<IPython.core.display.HTML object>

In [122]:
pd.set_option('max_colwidth', 400)
medcomplaints = filtered_complaints[filtered_complaints['sub_product'] == 'Medical']
no = medcomplaints[medcomplaints['consumer_disputed']=='No']


doc2 = nlp(LONG_NEWS_ARTICLE)
displacy.render(doc2, style="ent")


In [130]:
no['complaint_what_happened_len']
ents_df = entsdf['ents'].explode().apply(pd.Series).drop(columns=[0]).merge(entsdf, right_index=True, left_index=True)


In [202]:
no['complaint_what_happened'][38277]

'credit collection agency Advance Bureau of Collection has caused a collection to appear on my credit report for no reason, i did not receive any correspondence, letters or calls from this company before this happened. i was notified of this collection on my report by a ID theft monitoring company XXXX which monitors changes to my credit report. The company that submitted this collection action on my report is Advance Bureau of Collection XXXX XXXX XXXX XXXX GA XXXX. I asked the my telephone call to remove it immediately but they said it would not be done at this time and not for 3 months or more. This company stated that a computer error caused the collection to be placed on my XXXX report in error. I talked to XXXX XXXX and a lady named XXXX XXXX to get assistance and they were not willing to write me a letter statement of error or provide me any other paperwork stating it was their error. this company should not be allowed to tarnish my credit score which was XXXX to XXXX before thi

In [159]:
def get_ent_text(df):
    l = []
    for i, dfr in df.iterrows():
        start = int(dfr['start'])
        stop = int(dfr['end'])
      
        
        text = dfr['text']
        l.append(text[start:stop])
    return l

#ents_df.apply(lambda row: get_ent_text(row), axis=0)
ents_df = ents_df.dropna(subset=['start', 'end'])
ents_df['ent_text'] = get_ent_text(ents_df)

edf = ents_df[ents_df['ent_text'].map(lambda x: False if 'XX' in x else True)][['text', 'ent_text', 'label', 'end', 'start']]
edf['label'].value_counts()

ORG         218
CARDINAL    125
DATE         98
MONEY        81
ORDINAL      27
TIME         19
LAW          18
GPE          14
PERCENT       8
PRODUCT       7
PERSON        4
EVENT         3
LANGUAGE      2
NORP          2
QUANTITY      1
LOC           1
Name: label, dtype: int64

In [192]:
edf.merge(filtered_complaints, right_on='consumer_complaint_')

,end,label,start,text,ents,sents,tokens,ent_text
0,25.0,ORG,15.0,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/XX/XXXX I requested that Experian send me a copy of the verifiable proof they have on file showing that the XXXX account they have listed on my credit report is actually mine. On XX/XX/XXXX and XX/XX/XXXX, instead of sending me a copy of the verifiable proof that I requested, Experian sent me a statement which reads, "" The information you disputed ...","[{'start': 15, 'end': 25, 'label': 'ORG'}, {'start': 27, 'end': 37, 'label': 'ORG'}, {'start': 43, 'end': 46, 'label': 'ORG'}, {'start': 71, 'end': 79, 'label': 'ORG'}, {'start': 154, 'end': 158, 'label': 'ORG'}, {'start': 225, 'end': 235, 'label': 'ORG'}, {'start': 240, 'end': 250, 'label': 'ORG'}, {'start': 323, 'end': 331, 'label': 'ORG'}, {'start': 430, 'end': 438, 'label': 'ORG'}, {'start...","[{'start': 0, 'end': 46}, {'start': 46, 'end': 48}, {'start': 48, 'end': 53}, {'start': 54, 'end': 221}, {'start': 222, 'end': 429}, {'start': 430, 'end': 502}, {'start': 503, 'end': 687}, {'start': 688, 'end': 723}, {'start': 724, 'end': 915}, {'start': 916, 'end': 986}]","[{'id': 0, 'start': 0, 'end': 10, 'pos': 'ADV', 'tag': 'RB', 'dep': 'ROOT', 'head': 0}, {'id': 1, 'start': 10, 'end': 11, 'pos': 'PUNCT', 'tag': ',', 'dep': 'punct', 'head': 0}, {'id': 2, 'start': 12, 'end': 14, 'pos': 'ADP', 'tag': 'IN', 'dep': 'meta', 'head': 1}, {'id': 3, 'start': 15, 'end': 17, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'nmod', 'head': 7}, {'id': 4, 'start': 17, 'end': 18, 'pos'...",XX/XX/XXXX
0,37.0,ORG,27.0,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/XX/XXXX I requested that Experian send me a copy of the verifiable proof they have on file showing that the XXXX account they have listed on my credit report is actually mine. On XX/XX/XXXX and XX/XX/XXXX, instead of sending me a copy of the verifiable proof that I requested, Experian sent me a statement which reads, "" The information you disputed ...","[{'start': 15, 'end': 25, 'label': 'ORG'}, {'start': 27, 'end': 37, 'label': 'ORG'}, {'start': 43, 'end': 46, 'label': 'ORG'}, {'start': 71, 'end': 79, 'label': 'ORG'}, {'start': 154, 'end': 158, 'label': 'ORG'}, {'start': 225, 'end': 235, 'label': 'ORG'}, {'start': 240, 'end': 250, 'label': 'ORG'}, {'start': 323, 'end': 331, 'label': 'ORG'}, {'start': 430, 'end': 438, 'label': 'ORG'}, {'start...","[{'start': 0, 'end': 46}, {'start': 46, 'end': 48}, {'start': 48, 'end': 53}, {'start': 54, 'end': 221}, {'start': 222, 'end': 429}, {'start': 430, 'end': 502}, {'start': 503, 'end': 687}, {'start': 688, 'end': 723}, {'start': 724, 'end': 915}, {'start': 916, 'end': 986}]","[{'id': 0, 'start': 0, 'end': 10, 'pos': 'ADV', 'tag': 'RB', 'dep': 'ROOT', 'head': 0}, {'id': 1, 'start': 10, 'end': 11, 'pos': 'PUNCT', 'tag': ',', 'dep': 'punct', 'head': 0}, {'id': 2, 'start': 12, 'end': 14, 'pos': 'ADP', 'tag': 'IN', 'dep': 'meta', 'head': 1}, {'id': 3, 'start': 15, 'end': 17, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'nmod', 'head': 7}, {'id': 4, 'start': 17, 'end': 18, 'pos'...",XX/XX/XXXX
0,46.0,ORG,43.0,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/XX/XXXX I requested that Experian send me a copy of the verifiable proof they have on file showing that the XXXX account they have listed on my credit report is actually mine. On XX/XX/XXXX and XX/XX/XXXX, instead of sending me a copy of the verifiable proof that I requested, Experian sent me a statement which reads, "" The information you disputed ...","[{'start': 15, 'end': 25, 'label': 'ORG'}, {'start': 27, 'end': 37, 'label': 'ORG'}, {'start': 43, 'end': 46, 'label': 'ORG'}, {'start': 71, 'end': 79, 'label': 'ORG'}, {'start': 154, 'end': 158, 'label': 'ORG'}, {'start': 225, 'end': 235, 'label': 'ORG'}, {'start': 240, 'end': 250, 'label': 'ORG'}, {'start': 323, 'end': 331, 'label': 'ORG'}, {'start': 430, 'end': 438, 'label': 'ORG'}, {'start...","[{'start': 0, 'end': 46}, {'start': 46, 'end': 48}, {'start': 48, 'end': 53}, {'start': 54, 'end': 221}, {'star

In [162]:
edf[edf['label'].isin(['LAW'])]#, 'GPE', 'MONEY', 'TIME', 'NORP', 'LOC', ])]

,text,ent_text,label,end,start
0,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/XX/XXXX I requested that Experian send me a copy of the verifiable proof they have on file showing that the XXXX account they have listed on my credit report is actually mine. On XX/XX/XXXX and XX/XX/XXXX, instead of sending me a copy of the verifiable proof that I requested, Experian sent me a statement which reads, "" The information you disputed ...",the Fair Credit Reporting Act,LAW,687.0,658.0
5,"Dear Experian/XXXX XXXX XXXX listed below, which appears on my credit report, does not relate to any transaction ( s ) that I have made. It is the result of identity theft. Please block this information from my credit report, pursuant to section 605B of the Fair Credit Reporting Act, and send the required notifications to all furnishers of this information. Enclosed are the following : A copy...",the Fair Credit Reporting Act,LAW,284.0,255.0
5,"Dear Experian/XXXX XXXX XXXX listed below, which appears on my credit report, does not relate to any transaction ( s ) that I have made. It is the result of identity theft. Please block this information from my credit report, pursuant to section 605B of the Fair Credit Reporting Act, and send the required notifications to all furnishers of this information. Enclosed are the following : A copy...",the Fair Credit Reporting Act,LAW,445.0,416.0
18,"According to the Fair Credit Reporting Act, Section 609 ( a ) ( 1 ) ( A ), you are required by federal law to verify -through the physical verification of the original signed consumer contract -any and all accounts you post on a credit report. Otherwise, anyone paying for your reporting services could fax, mail or email in a fraudulent account. I demand to see Verifiable Proof ( an original C...",the Fair Credit Reporting Act,LAW,42.0,13.0
19,"XX/XX/19 XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, GA XXXX XXXX XXXX XXXX, XXXX XXXX XXXX XXXX XXXX XXXX, GA XXXX To Whom It May Concern : I have obtain copies of the allege bankruptcies in my name : ( XXXX ; XXXX ) But after carefully reviewing them I notices that the signature appearing in them is not mine, I'm sending a notarize letter with my signature to verify that these items you have li...",section 1681,LAW,571.0,559.0
19,"XX/XX/19 XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, GA XXXX XXXX XXXX XXXX, XXXX XXXX XXXX XXXX XXXX XXXX, GA XXXX To Whom It May Concern : I have obtain copies of the allege bankruptcies in my name : ( XXXX ; XXXX ) But after carefully reviewing them I notices that the signature appearing in them is not mine, I'm sending a notarize letter with my signature to verify that these items you have li...",the Fair Credit Reporting Act,LAW,616.0,587.0
30,"Accounts added and/or created unbeknownst to me Notice to whom it be of concern, It is YOU I hold accountable and accuse of complicity in exercise of malevolent did-compliance. I hereby mark you as likely one that I might one day view in civil discord as the "" would be plaintiffs '', yet I declare to you your unjust claim is to be met with deliberate and determined resistance and counteraction...",HIPAA PRIVACY RULE,LAW,2351.0,2333.0
32,I have been trying to get my Private Mortgage Insurance Removed from my mortgage since XX/XX/XXXX when my mortgage dropped below 80 % loan to value. Last year my mortgage was sold from XXXX XXXX ( Under mortgage # XXXX ) to Ditech Mortgage ( account # XXXX ). I reached out to Ditech via a email ( after being told to do so via phone representative ) request to remove my PMI on mortgage on XX/X...,"an "" Automatic Termination of PMI ''",LAW,571.0,535.0
32,I have been trying to get my Private Mortgage Insurance Removed from my mortgage since XX/XX/XXXX when my mortgage dropped below 80 % loan to value. Last year my mortgage was sold from XXXX XXXX ( Under mortgage # XXXX ) to Ditech Mortgage ( account # XXXX ). I reached out to Ditech via a email ( after being told to do so via phone representative ) request to remove my PMI on mort

In [ ]:
doc2 = nlp(ents_df['text'][])
displacy.render(doc2, style="ent")

In [177]:
ents_df['text'][94].unique()

array(['RE : XXXX, Transunion, XXXX, XXXX XXXX XXXX and XXXX Dates, XX/XX/XXXX, XX/XX/XXXX, XX/XX/XXXX, and XX/XX/XXXX XXXX : - XXXX [ Amount : {$550.00} ], Creditor states account is open, not true, lack of signed contract - XXXX [ Amount : {$660.00} ] Creditor states account is open, not true, lack of signed contract - XXXX [ Amount : {$730.00} ] Creditor states account is open, not true, lack of signed contract - XXXX XXXX XXXX XXXX [ Amount : {$400.00} ] lack of signed contract - XXXX XXXX XXXX [ Amount : {$1100.00} ], reporting three times on my report paid and unpaid - XXXX XXXX XXXX XXXX [ Amount : XXXX ], lack of signed contract, and never had account with this firm [ has not provided contract of debt owed ] - XXXX XXXX XXXX [ Amount : {$890.00} ], according to HIPPA law, did not give hospital nor collection company, permission to release my health records, - XXXX XXXX, XXXX XXXX XXXX [ Amount : {$670.00} ] - XXXX XXXX XXXX XXXX [ Amount : XXXX ], never had an account with comp

Experian                          20
3                                 19
two                               13
FCRA                              13
2                                 12
                                  ..
14000.00                           1
only 30 days                       1
Covid                              1
Shellpoint Mortgaging Services     1
270.00                             1
Name: ent_text, Length: 314, dtype: int64

In [210]:
def get_verbs(doc):
    """
    param:
    returns: verb token objects
    """
    verbs = set()

    for possible_subject in doc:
        if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
            verbs.add(possible_subject.head)
            
            
    return verbs

docs = nlp.pipe(TEXTS)

verbs_list = [get_verbs(doc) for doc in docs]

NameError: name 'TEXTS' is not defined

In [ ]:
ents_df